In [4]:
import os
OUTPUT_DIR = os.path.join("lips","Outputs")
INPUT_DIR = os.path.join("lips", "Outputs", "Data", "test")
BENCHMARK_DIR = os.path.join("lips", "Outputs", "Benchmark 1")

In [17]:
import numpy as np
from pprint import pprint
from lips.dataset import PowerGridDataSet
from lips.simulators import DCApproximation
from lips.benchmark import Benchmark
from lips.evaluation import Evaluation
from lips.simulators import FullyConnectedNN
from pprint import pprint

In [3]:
# Run the cell below to produce a table of contents 
res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

## Data Generation
To generate the datasets, we use `PowerGridDataSet` class which is a sub-class of a more general `DataSet` class and is designed specifically for power grid data. It allows to generate one dataset at a time. We keep the DataSet class (therefore the platform) as generic as possible for its use in other research and industrial domains. 

In [3]:
dataset = PowerGridDataSet(env_name="l2rpn_case14_sandbox")

In [7]:
test_data = dataset.generate(nb_samples=int(1024) * 128,
                             tag="test",
                             val_regex=".*99[0-9].*",
                             use_lightsim_if_available=True,
                             reference_number=0,
                             agent_generator_name="random_nn1",
                             agent_parameters={"p": 0.5},
                             skip_gameover=True,
                             env_seed=1234,
                             agent_seed=14,
                             verbose=True
                            )

C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:33: UserWarning: There were some Nan in the pp_net.trafo["tap_neutral"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_neutral\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:41: UserWarning: There were some Nan in the pp_net.trafo["tap_step_percent"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_percent\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:46: UserWarning: There were some Nan in the pp_net.trafo["tap_pos"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_pos\"], they have been replaced by 0")
100%|█████████████████████████████████████████████████████████████████████████| 131072

#### save the generated dataset
<span style="color:red">In this version, I have stored a list of Grid2op `Observations`, but it takes a lot of space on disk !!! </span>

In [10]:
dataset.save(path=OUTPUT_DIR)

#### Load the generated dataset 

In [5]:
test_data = dataset.load(INPUT_DIR)

C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:33: UserWarning: There were some Nan in the pp_net.trafo["tap_neutral"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_neutral\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:41: UserWarning: There were some Nan in the pp_net.trafo["tap_step_percent"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_percent\"], they have been replaced by 0")
C:\Users\milad.leyli-abadi\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:46: UserWarning: There were some Nan in the pp_net.trafo["tap_pos"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_pos\"], they have been replaced by 0")


## Loading and evaluating a Simulator

### Physical Simulator (DC approximation)
To assess the performance of a physical solver, one can use the DCApproximation available in `simulators` module. As opposed to augmented simulators, it does not require to be learned. Hence, only test dataset should be used to evaluate its performance. Various variables could be used as inputs and outputs of this solver.

In [6]:
dc_app = DCApproximation(dataset.env._init_grid_path,
                         name="dc_approx",
                         attr_x=("prod_p", "prod_v", "load_p", "load_q", "topo_vect"),  # input that will be given to the proxy
                         attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_or", "q_ex", "prod_q", "load_v", "v_or", "v_ex"),
                        )

DC approximator should be initialized with generated data. 

In [8]:
# I have reduced the size of experimentation, as it takes long time to be excuted for DC approximation
dc_app.init(test_data[:10000])

Once, intitialized, it allows to infer the output variables from the input ones using its `predict` function.

In [9]:
predictions, observations, predict_time = dc_app.predict(save_path=BENCHMARK_DIR)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [07:47<00:00, 21.37it/s]


Observations and Predictions are saved successfully !


## Evaluation metrics
The `Evaluation` class allows to evaluate the performance of different simulators with respect to various criteria arosed from various domains. 
- Physic compliance
- Machine Learning metrics
- Generalization
- Readiness (not yet implemented)

In this new version of platform, one should activate or deactivate the verifications required for a specific benchmark, before passing an object of this class to the `Benchmark` class.

we start by creating an object of the Evaluation class as follows:

In [11]:
evaluation = Evaluation()

Then, a dictionary of Metric Activation is provided using `get_active_dict()` function : 

In [12]:
active_dict = evaluation.get_active_dict()

In [18]:
pprint(active_dict)

{'evaluate_ML': False,
 'evaluate_adaptability': False,
 'evaluate_physic': {'verify_EL': False,
                     'verify_KCL': False,
                     'verify_LCE': False,
                     'verify_current_eq': False,
                     'verify_current_pos': False,
                     'verify_loss_pos': False,
                     'verify_predict_disc': False,
                     'verify_voltage_pos': False},
 'evaluate_readiness': False}


#### Activate the required verifications 

For example, if we wish for benchmark 1 only test the ML metrics and to verify the positivity of current values, one possible way to active them is the following :

In [19]:
active_dict["evaluate_ML"] = True
active_dict["evaluate_physic"]["verify_current_pos"] = True
evaluation.set_active_dict(active_dict)

In [20]:
pprint(evaluation.get_active_dict())

{'evaluate_ML': True,
 'evaluate_adaptability': False,
 'evaluate_physic': {'verify_EL': False,
                     'verify_KCL': False,
                     'verify_LCE': False,
                     'verify_current_eq': False,
                     'verify_current_pos': True,
                     'verify_loss_pos': False,
                     'verify_predict_disc': False,
                     'verify_voltage_pos': False},
 'evaluate_readiness': False}


It can be verified as above, that these two tests are activated for the Benchmark.

## Benchmark
Once a dataset has been generated, a simulator has been trained and predicted the required results and an evaluator with required metrics is parameterized, the `Benchmark` class allows to assess the performance of the simulator with respect to the indicated metrics and criteria. 

As it can be seen below, the Benchmark class, takes as input an object of the following classes:

- PowerGridDataSet
- DCApproximation
- Evaluation

In [32]:
benchmark_dc = Benchmark("Risk assessment", dataset, dc_app, evaluation, save_path=BENCHMARK_DIR)

The prediction time of the corresponding simulator could be accessed rapidly.

In [33]:
benchmark_dc.prediction_time

205.00069546699524

Now, we can evaluate the simulator using all the indicated metrics in the previous section by activating the metrics. The choice allows to consider the real observations for physic compliance verifications. However, for Machine Learning metrics, the verification does not change by this parameter.

In [34]:
benchmark_dc.evaluate_simulator(choice="real",
                                EL_tolerance=0.04,
                                LCE_tolerance=1e-3,
                                KCL_tolerance=1e-2,
                                active_flow=True,
                                save_path=None
                               )

************* Basic verifier *************
Current positivity check passed for origin side !
----------------------------------------------
Current positivity check passed for extremity side !
----------------------------------------------


All the metrics could be accessed via the member variables of Benchmark class as follows : 

#### ML metrics

In [39]:
print("The list of all the available metrics")
pprint(benchmark_dc.metrics_ML.keys())
print("********************")
print("MAE_avg")
pprint(benchmark_dc.metrics_ML["MAE_avg"])
print("MAPE")
pprint(benchmark_dc.metrics_ML["mape_avg"])
print("MAPE90")
pprint(benchmark_dc.metrics_ML["mape90"])

The list of all the available metrics
dict_keys(['MSE_avg', 'MAE_avg', 'NRMSE_avg', 'pearson_r_avg', 'mape_avg', 'rmse_avg', 'MSE', 'MAE', 'NRMSE', 'pearson_r', 'mape', 'rmse', 'mape90', 'MAE90'])
********************
MAE_avg
{'a_ex': 91.23420715332031,
 'a_or': 76.09380340576172,
 'load_v': 0.9383689761161804,
 'p_ex': 0.8885318040847778,
 'p_or': 0.9869451522827148,
 'prod_q': 18.775745391845703,
 'q_ex': 6.444573879241943,
 'q_or': 6.574471473693848,
 'v_ex': 1.0005929470062256,
 'v_or': 0.6585171222686768}
MAPE
{'a_ex': 0.20803854516880946,
 'a_or': 0.22168532480028338,
 'load_v': 0.0335074732580896,
 'p_ex': 0.08478020147489072,
 'p_or': 0.08923333471048199,
 'prod_q': 0.7642740689280486,
 'q_ex': 1.0,
 'q_or': 1.0,
 'v_ex': 0.03471973123270375,
 'v_or': 0.022346339484299878}
MAPE90
{'a_or': 0.14264498503758033,
 'p_or': 0.04650562424478667,
 'q_or': 1.0,
 'v_or': 0.019656345361385094}


#### Physic compliances 

In [40]:
benchmark_dc.metrics_physics

{'BasicVerifications': {'currents': {}}}

as there is no violation of current values, this metric does not contain any value.

The Physic compliance verification could also be performed for predictions.

In [41]:
benchmark_dc.evaluate_simulator(choice="predictions",
                                EL_tolerance=0.04,
                                LCE_tolerance=1e-3,
                                KCL_tolerance=1e-2,
                                active_flow=True,
                                save_path=None
                               )

************* Basic verifier *************
Current positivity check passed for origin side !
----------------------------------------------
Current positivity check passed for extremity side !
----------------------------------------------


As it can be seen there is no violation for the positivity of current values.

# <span style="color:red">Critical evaluation</span>

Here, I activate the EL, LCE and KCL metrics from physic compliance category.

In [42]:
active_dict = evaluation.get_active_dict()

In [43]:
active_dict

{'evaluate_ML': True,
 'evaluate_physic': {'verify_current_pos': True,
  'verify_voltage_pos': False,
  'verify_loss_pos': False,
  'verify_predict_disc': False,
  'verify_current_eq': False,
  'verify_EL': False,
  'verify_LCE': False,
  'verify_KCL': False},
 'evaluate_adaptability': False,
 'evaluate_readiness': False}

In [44]:
active_dict["evaluate_physic"]["verify_EL"] = True
active_dict["evaluate_physic"]["verify_LCE"] = True
active_dict["evaluate_physic"]["verify_KCL"] = True

In [45]:
evaluation.set_active_dict(active_dict)

In [47]:
pprint(evaluation.get_active_dict())

{'evaluate_ML': True,
 'evaluate_adaptability': False,
 'evaluate_physic': {'verify_EL': True,
                     'verify_KCL': True,
                     'verify_LCE': True,
                     'verify_current_eq': False,
                     'verify_current_pos': True,
                     'verify_loss_pos': False,
                     'verify_predict_disc': False,
                     'verify_voltage_pos': False},
 'evaluate_readiness': False}


### <span style="color:red">Now Evaluating these criteria using Benchmark class</span>

In [48]:
benchmark_dc = Benchmark("Risk assessment", dataset, dc_app, evaluation, save_path=BENCHMARK_DIR)

In [49]:
benchmark_dc.evaluate_simulator(choice="real",
                                EL_tolerance=0.04,
                                LCE_tolerance=1e-3,
                                KCL_tolerance=1e-2,
                                active_flow=True,
                                save_path=None
                               )

************* Basic verifier *************
Current positivity check passed for origin side !
----------------------------------------------
Current positivity check passed for extremity side !
----------------------------------------------
************* Check loss *************
Verification is done without any violation !
************* Check Energy Conservation *************
Number of failed cases is 0 and the proportion is 0.000% : 
************* Check kirchhoff's current law *************
0.00% not verify the Kirchhoff's current law at 0.01 tolerance


There is no problem using Real Observations

In [50]:
benchmark_dc.evaluate_simulator(choice="predictions",
                                EL_tolerance=0.04,
                                LCE_tolerance=1e-3,
                                KCL_tolerance=1e-2,
                                active_flow=True,
                                save_path=None
                               )

************* Basic verifier *************
Current positivity check passed for origin side !
----------------------------------------------
Current positivity check passed for extremity side !
----------------------------------------------
************* Check loss *************
Verification is done without any violation !
************* Check Energy Conservation *************
Number of failed cases is 10000 and the proportion is 100.000% : 
************* Check kirchhoff's current law *************
7.14% not verify the Kirchhoff's current law at 0.01 tolerance


It can be seen here that, therea 100% of the cases that DC violates the Law of Conservation of Energy and 7% of the cases that it violates the Kirchoff's current law.

### <span style="color:red">I dont think that it could be a problem from the implementation point of view !</span>